In [1]:
#default_exp train_slice

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [ ]:
#export
class OpenCTs:
    def __init__(self, path): 
        self.fn = get_pil_fn(path)
        self.tt = ToTensor()
    def __call__(self, item):
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        return TensorCTScan(torch.stack(xs))

In [ ]:
#export
def pad_batch(x, pad_to=None, value=0):
    bs_pad = pad_to-x.shape[0]
    pad = [0]*len(x.shape)*2
    pad[-1] = bs_pad
    return F.pad(x, pad=pad, value=value)

In [ ]:
#export
class TfmSlice:
    def __init__(self,df,open_fn,pad_to=None):
        self.open_fn = open_fn
        self.df = df
    
    def x(self, sid):
        sids = self.df.SOPInstanceUID[sid].values
        x = self.open_fn(sids)
        if self.pad_to is None: return x
        t = type(x)
        return t(pad_batch(x, pad_to=self.pad_to))
    
    def y(self, sid): 
        vals = self.df.loc[sid,htypes].values
        if self.pad_to is not None: 
            vals = pad_batch(tensor(vals), pad_to=self.pad_to, value=-1)
        return TensorMultiCategory(vals).float()

In [ ]:
def group_cv(idx, grps): return np.concatenate([grps[o] for o in range_of(grps) if o!=idx])

# column can also be PatientID
def split_data(df, cv_idx, grps, column):
    grp_cv = group_cv(cv_idx, grps)
    mask = df[column].isin(grp_cv)
    return idx[mask],idx[~mask]

In [ ]:
#export
def get_3d_dsrc(df, open_fn, grps=Meta.grps, cv_idx=0, column='SeriesInstanceUID',
               pad_to=None):
    df_series = df.reset_index().set_index(column).sort_values("ImagePositionPatient2")
    tfm = TfmSOP(df_series, open_fn, pad_to)
    sids = df_series.index.unique()
    
    s1 = np.where(np.in1d(sids, group_cv(cv_idx,grps)))[0]
    s2 = np.where(np.in1d(sids, grps[cv_idx]))[0]
    dsrc = DataSource(sids, [[tfm.x],[tfm.y]], splits=(s1,s2))
    return dsrc